### 리뷰 데이터 전처리

In [12]:
# import libraries
import pandas as pd
import numpy as np
import pickle
import os
import re
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Kkma
kkma = Kkma()
from collections import Counter


In [2]:
# load dataset
review = pd.read_csv('..\\reviewdataset\\review_raw.csv')
review[:3] # check

,직무,고용 현황,근무지,날짜,별점,요약,장점,단점,경영진에게 바라는 점
0,금융/재무,현직원,서울,NaN,5.0,"""수평적문화 , 워라밸 ,복지포인트 많이줌, 직원의견 긍정적으로 수렴.""",\n자동차보험 시장점유율이 빠르게 확보되는 중으로 성장성이 기대됨. 대물직원은 자회...,"\n일이 많고, 체계가 좀 부족하다는 점.차차 나아질거라고 봅니다.\n",\n앞으로 더 적극적으로 마케팅하고 내실을 다져 탄탄한 보험회사로 성장하길 기원합니...
1,디자인,현직원,서울,NaN,4.0,"""스타트업의 성격이 있어, 개인의 역할이 중요하다! 운영부터 구축까지 다양한 범위의...","\n연차를 개인의 스케줄에 맞게 사용할 수 있다. 부서마다 다르겠지만, 다양성을 존...",\n기본으로 제공되는 물품이 너무 적다.. 커피머신은 있지만 캡슐은 개인 구비해야하...,\n아직은 체계가 덜 잡혀 있는 느낌이 있습니다. 앞으로 더 체계가 잡혔으면 좋겠습...
2,금융/재무,현직원,서울,NaN,5.0,"""워라밸 균형, 시차 출근제, 수평적인 분위기, 영어이름, 젊은 사람들이 많음""",\n직급 또는 직책으로 부르지 않고 스스로 지은 영어이름으로 부르며 사무실 분위기는...,"\n부서간 알앤알이 명확하지않고, 명확하지 않은 만큼 일을 미루는 경향이 있기도 합...",\n작은 회사인만큼 직원들의 말에 귀기울여주기 바랍니다.\n


In [3]:
# types of 직무
review['직무'].unique()

array(['금융/재무', '디자인', '기획/경영', '개발', '마케팅/시장조사', '전문직', '서비스/고객지원',
       '데이터', '인사/총무', '교육', '영업/제휴', '생산/제조', '미디어/홍보', '유통/무역', '기타',
       '엔지니어링', nan, '연구개발', '생산관리/품질관리', '특수계층/공공',
       '\n          현직원\n      ', '\n          전직원\n      ', '법률/법무',
       '의약'], dtype=object)

#### 결측치 제거

In [4]:
# drop na
review = review[(review['직무']!='\n          전직원\n      ')&(review['직무']!='\n          현직원\n      ')&(review['직무'].notna())]
len(review)

47827

In [5]:
# check
review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47827 entries, 0 to 48469
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   직무           47827 non-null  object 
 1   고용 현황        47827 non-null  object 
 2   근무지          47827 non-null  object 
 3   날짜           47266 non-null  object 
 4   별점           47827 non-null  float64
 5   요약           47827 non-null  object 
 6   장점           47827 non-null  object 
 7   단점           47827 non-null  object 
 8   경영진에게 바라는 점  47827 non-null  object 
dtypes: float64(1), object(8)
memory usage: 3.6+ MB


#### 클렌징

In [6]:
# 장점
review['장점'] = review['장점'].str.replace("\n"," ")
review['장점'] = review['장점'].str.replace("\r"," ")
review['장점'] = review['장점'].str.replace("\t"," ")
review['장점'] = review['장점'].str.replace(",","")

# 단점
review['단점'] = review['단점'].str.replace("\n"," ")
review['단점'] = review['단점'].str.replace("\r"," ")
review['단점'] = review['단점'].str.replace("\t"," ")
review['단점'] = review['단점'].str.replace(",","")

# 요약
review['요약'] = review['요약'].str.replace("\n"," ")
review['요약'] = review['요약'].str.replace("\r"," ")
review['요약'] = review['요약'].str.replace("\t"," ")
review['요약'] = review['요약'].str.replace(",","")

In [7]:
# regular expression
review['장점']= review['장점'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)
review['단점']= review['단점'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

In [8]:
# # okt normalization (poor performance)
# for i, review in enumerate(review_df['장점']):
#     review.loc[i,'장점'] =  okt.normalize(review)

# for i, review in enumerate(review_df['단점']):
#     review.loc[i,'단점'] =  okt.normalize(review)

#### 장단점 컬럼 생성

In [9]:
# new 장단점 column by combining 장점 and 단점 col
review['장단점'] = review['장점']+review['단점']

#### 테마 키워드 추출

In [10]:
# check
# okt.tagset

In [13]:
# extract theme keywords (noun) by 장단점 data
words_noun = []

for rvw in review['장단점']:
    pos = okt.nouns(rvw) # only Nouns
    words_noun.extend(pos)
# print(words_noun)

# # of elements
count_noun = Counter(words_noun)

In [14]:
# top 100 keywords
dict({key: value for key, value in sorted(count_noun.items(), key = lambda item: item[1], reverse=True)[:100]})

{'회사': 25773,
 '수': 21920,
 '업무': 21750,
 '사람': 21315,
 '일': 17547,
 '직원': 15281,
 '분위기': 14210,
 '복지': 13974,
 '연차': 13341,
 '것': 11878,
 '팀': 11674,
 '야근': 11318,
 '함': 10979,
 '연봉': 10643,
 '눈치': 7846,
 '사용': 7766,
 '문화': 7066,
 '기업': 7014,
 '생각': 6963,
 '근무': 6308,
 '체계': 5958,
 '편': 5945,
 '시간': 5629,
 '등': 5577,
 '장점': 5485,
 '지원': 5373,
 '때': 5347,
 '부서': 5335,
 '가능': 5109,
 '비': 4819,
 '퇴근': 4680,
 '대한': 4588,
 '및': 4383,
 '수평': 4283,
 '그': 4233,
 '워': 4150,
 '때문': 4111,
 '제공': 4099,
 '임': 4058,
 '대표': 3965,
 '곳': 3951,
 '경우': 3900,
 '자유': 3809,
 '말': 3791,
 '단점': 3772,
 '경험': 3753,
 '위': 3747,
 '매우': 3681,
 '점': 3557,
 '환경': 3534,
 '퇴사': 3446,
 '사업': 3430,
 '정도': 3407,
 '개인': 3402,
 '성장': 3375,
 '보고': 3369,
 '출퇴근': 3282,
 '개발': 3273,
 '더': 3255,
 '능력': 3206,
 '안': 3169,
 '업계': 3064,
 '내': 3056,
 '급여': 3046,
 '다른': 3044,
 '사내': 3036,
 '느낌': 3024,
 '본인': 2996,
 '이': 2913,
 '편이': 2904,
 '경력': 2904,
 '기회': 2872,
 '월급': 2872,
 '휴가': 2870,
 '프로젝트': 2855,
 '출근': 2819,
 '위치': 2722,
 '

In [15]:
# save & read as a pickle file

# save
with open('..\\reviewdataset\\keywords.pkl', 'wb') as tf:
	pickle.dump(count_noun, tf)

# read
with open('..\\reviewdataset\\keywords.pkl', 'rb')as tf:
	count_noun = pickle.load(tf)

##### 테마 분류
- 워라벨: 업무, 일, 야근, 근무, 시간, 퇴근, 자유, 경험, 성장, 능력, 기회, 강도
- 회사 분위기: 직원, 분위기, 팀, 눈치, 문화, 체계, 수평, 환경, 조직, 제도
- 복지: 복지, 연차, 휴가, 식대
- 연봉: 연봉, 급여, 월급, 돈
- 회사 위치: 출퇴근, 위치 (개인마다 차이 있을 수도..)

#### 직무별 긍정/부정별 데이터 나누기

##### +a
* ~~'음'으로 끝나는 부분에서 split
* 맞춤법 교정(https://velog.io/@acdongpgm/NLP-%ED%95%9C%EA%B5%AD%EC%96%B4-%EA%B5%AC%EC%96%B4%EC%B2%B4%EB%8C%80%ED%99%94%EC%B2%B4-%EB%A7%9E%EC%B6%A4%EB%B2%95-%EA%B5%90%EC%A0%95%EA%B8%B0Korean-typos-corrector-ET5-Text2Text-Generation)

In [16]:
# make a folder if it's not exist yet
def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [18]:
# make a folder to save dataset
folderpath = '..\\reviewdataset\\review_posneg_byjob'
makedirs(folderpath)

# job list
job = ['금융/재무', '디자인', '기획/경영', '개발', '마케팅/시장조사', '전문직', '서비스/고객지원',
       '데이터', '인사/총무', '교육', '영업/제휴', '생산/제조', '미디어/홍보', '유통/무역', '기타',
       '엔지니어링', '연구개발', '생산관리/품질관리', '특수계층/공공', '법률/법무', '의약']

for i in range(len(job)):
    job_name = job[i]
    job_pos = []
    job_neg = []
    job_pos_sen = []
    job_neg_sen = []
    job_df = review[review['직무']==job[i]]

    # positive ###
    job_pos.extend(job_df['장점'])
    for sentence in job_pos:
        if re.search(r'[0-9].', sentence):
            job_pos_sen.extend(re.split(r'[0-9].', sentence))
        elif '.' in sentence:
            job_pos_sen.extend(sentence.split('.'))
        elif '/' in sentence:
            job_pos_sen.extend(sentence.split('/'))
        else:
            job_pos_sen.append(sentence)
    # save as pickle files
    with open('{}\\{}_pos.pkl'.format(folderpath, job_name.replace('/','')), 'wb') as tf:
        pickle.dump(job_pos_sen, tf)

    # negative ###
    job_neg.extend(job_df['단점'])
    for sentence in job_neg:
        if re.search(r'[0-9].', sentence):
            job_neg_sen.extend(re.split(r'[0-9].', sentence))
        elif '.' in sentence:
            job_neg_sen.extend(sentence.split('.'))
        elif '/' in sentence:
            job_neg_sen.extend(sentence.split('/'))
        else:
            job_neg_sen.append(sentence)
    # save as pickle files
    with open('{}\\{}_neg.pkl'.format(folderpath, job_name.replace('/','')), 'wb') as tf:
        pickle.dump(job_neg_sen, tf)

#### 직무 안에서 테마별 데이터 나누기

In [19]:
# dictionaries by theme
balance = ['업무', '야근', '근무', '시간', '퇴근', '자유', '경험', '성장', '능력', '기회', '강도']
environment = ['직원', '분위기', '팀', '눈치', '문화', '체계', '수평', '환경', '조직', '제도']
benefit = ['복지', '연차', '휴가', '식대','자율출퇴근']
salary = ['연봉', '급여', '월급', '돈']

In [20]:
# make a folder to save dataset
folderpath = '..\\reviewdataset\\review_posneg_theme_byjob'
makedirs(folderpath)

## pkl file list
path = '..\\reviewdataset\\review_posneg_byjob\\'
file_list = os.listdir(path)
file_list_name = [file for file in file_list if file.endswith('.pkl')]

# read pkl file
for file in file_list_name:
    filename = file.split('.pkl')[0]

    with open('{}{}'.format(path, file), 'rb')as tf:
	    sentence_list = pickle.load(tf)
    tf.close()

    # remove ''
    sentence_list = [v for v in sentence_list if v]

    # make a list for sentences by theme (give 1st value as '')
    balance_sen = ['']
    environment_sen = ['']
    benefit_sen = ['']
    salary_sen = ['']

    for sentence in sentence_list:
        # balance
        for bal in balance: # dictionary
            if bal in sentence:
                if sentence != balance_sen[-1]:
                    balance_sen.append(sentence)
                else:
                    continue
        
        # environment
        for env in environment: # dictionary
            if env in sentence:
                if sentence != environment_sen[-1]:
                    environment_sen.append(sentence)
                else:
                    continue
        
        # benefit
        for ben in benefit: # dictionary
            if ben in sentence:
                if sentence != benefit_sen[-1]:
                    benefit_sen.append(sentence)
                else:
                    continue
        
        # salary
        for sal in salary: # dictionary
            if sal in sentence:
                if sentence != salary_sen[-1]:
                    salary_sen.append(sentence)
                else:
                    continue
    
    # pop the first value('')
    balance_sen.pop(0)
    environment_sen.pop(0)
    benefit_sen.pop(0)
    salary_sen.pop(0)

    # pickle로 저장
    with open('{}\\{}_balance.pkl'.format(folderpath, filename), 'wb') as tf:
        pickle.dump(balance_sen, tf)
    
    with open('{}\\{}_environment.pkl'.format(folderpath, filename), 'wb') as tf:
        pickle.dump(environment_sen, tf)
    
    with open('{}\\{}_benefit.pkl'.format(folderpath, filename), 'wb') as tf:
        pickle.dump(benefit_sen, tf)
    
    with open('{}\\{}_salary.pkl'.format(folderpath, filename), 'wb') as tf:
        pickle.dump(salary_sen, tf)


In [25]:
# check
with open('..\\reviewdataset\\review_posneg_byjob\\개발_pos.pkl', 'rb')as tf:
	sentence_list = pickle.load(tf)
tf.close()
sentence_list[:10]

[' 부바부지만 워라벨이 좋습니다  일정조율이나 이런 부분도 먼저 물어보고 조율을 합니다  물론 급한 일정들은 늘 있지만 그래도 최대한 워라벨을 보장해 주려고하는 느김입니다  ',
 ' 성장성이 높은 기업이며대기업의 장점 과 스타트업 장점이 있는곳지하철이 연결됨 초고층뷰대표님 마인드가 직원들을 많이 생각한다고 느껴짐 ',
 ' 아직 대외적으로는 이미지가 좋은듯 하다  신기술로 뭔가 인슈어테크 하는것 처럼 보임 ',
 ' 눈치 안보고 연차 쓸수 있음 넉넉한 복지포인트 및 체력단련 휴가 지하철역에서 바로 회사로 출퇴근 할 수 있음 ',
 ' 수평적인 분위기 워라벨 자유로운 연차사용 쾌적한 사무실 역세권인 회사 입지 ',
 ' 이 회사으이 장점은 일단 해볼수있는게 많다는거다  본인이 하고자하면 할 수 있는게 많다  물론 그 안에서 약간의 정치가 필요하다 ',
 ' 영어이름 워라벨 급여 높은편 복지포인트 숙소비 체력단련휴가 ',
 ' 등 좋은점은 꽤 됨 ',
 ' 연차를 눈치안보고 쓸 수 있고 연차 추가로 ',
 ' 더줌그거말고 별다른 복지없음 ']